In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rectal_Cancer"
cohort = "GSE40492"

# Input paths
in_trait_dir = "../../input/GEO/Rectal_Cancer"
in_cohort_dir = "../../input/GEO/Rectal_Cancer/GSE40492"

# Output paths
out_data_file = "../../output/preprocess/Rectal_Cancer/GSE40492.csv"
out_gene_data_file = "../../output/preprocess/Rectal_Cancer/gene_data/GSE40492.csv"
out_clinical_data_file = "../../output/preprocess/Rectal_Cancer/clinical_data/GSE40492.csv"
json_path = "../../output/preprocess/Rectal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I analyzed the patient data for this rectal cancer dataset. The corrections focus on properly handling the clinical data without relying on a pre-existing CSV file.

```python
# 1. Gene Expression Data Availability
# Based on the background info, this dataset contains gene expression data for rectal cancer patients.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Rectal Cancer)
# Looking at the clinical features, we can use pathological lymph node status after treatment
# From row 9: 'pathological lymphnode status after neoadjuvant treatment and surgery (0,1,2 - according to uicc tnm classification)'
trait_row = 9

# For age
# Age is available in row 1
age_row = 1

# For gender
# Gender is available in row 2 as 'Sex'
gender_row = 2

# 2.2 Data Type Conversion Functions

def convert_trait(value_str):
    """Convert lymph node status to binary value.
    0 = No positive lymph nodes, 1 = Positive lymph nodes"""
    if value_str is None or 'NA' in value_str:
        return None
    
    # Extract the value after the colon
    if ':' in value_str:
        value = value_str.split(':', 1)[1].strip()
    else:
        value = value_str.strip()
    
    # Status 0 means no positive lymph nodes
    if value == '0':
        return 0
    # Status 1 or 2 means positive lymph nodes
    elif value in ['1', '2']:
        return 1
    else:
        return None

def convert_age(value_str):
    """Convert age to continuous value."""
    if value_str is None or 'NA' in value_str:
        return None
    
    # Extract the value after the colon
    if ':' in value_str:
        value = value_str.split(':', 1)[1].strip()
    else:
        value = value_str.strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value_str):
    """Convert gender to binary value. 0 = female, 1 = male"""
    if value_str is None or 'NA' in value_str:
        return None
    
    # Extract the value after the colon
    if ':' in value_str:
        value = value_str.split(':', 1)[1].strip().lower()
    else:
        value = value_str.strip().lower()
    
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We'll proceed only if trait_row is not None
if trait_row is not None:
    # Create the clinical data DataFrame from the sample characteristics dictionary
    sample_chars_dict = {0: ['dataset: Training', 'dataset: Validation'], 1: ['age: 55.5', 'age: 65.6', 'age: 62.6', 'age: 61.8', 'age: 52.1', 'age: 59.1', 'age: 70.6', 'age: 60.6', 'age: 55', 'age: 53.1', 'age: 58.5', 'age: 68.4', 'age: 58.8', 'age: 70', 'age: 77.5', 'age: 75.2', 'age: 76.3', 'age: 38.2', 'age: 61.1', 'age: 69.4', 'age: 54.2', 'age: 77.7', 'age: 57.4', 'age: 61.2', 'age: 56.5', 'age: 47', 'age: 62.7', 'age: 51.2', 'age: 73.2', 'age: 47.2'], 2: ['Sex: female', 'Sex: male'], 3: ['therapy: 5-FU + Oxaliplatin + RT', 'therapy: 5-FU + RT'], 4: ['surgery type: deep anterior resection (TAbdominoperineal resection (APR))', 'surgery type: Other', 'surgery type: Abdominoperineal excision (APE)', 'surgery type: NA', 'surgery type: Abdominoperineal resection (APR)'], 5: ['clinical tumor category (0,i,ii,iii,iv - according to uicc tnm classification): 3', 'clinical tumor category (0,i,ii,iii,iv - according to uicc tnm classification): 2', 'clinical tumor category (0,i,ii,iii,iv - according to uicc tnm classification): NA', 'clinical tumor category (0,i,ii,iii,iv - according to uicc tnm classification): 4'], 6: ['clinical lymphnode status (0,1 - according to uicc tnm classification): 1', 'clinical lymphnode status (0,1 - according to uicc tnm classification): 0', 'clinical lymphnode status (0,1 - according to uicc tnm classification): NA'], 7: ['clinical tumor stage (0,i,ii,iii,iv - according to uicc tnm classification): IV', 'clinical tumor stage (0,i,ii,iii,iv - according to uicc tnm classification): III', 'clinical tumor stage (0,i,ii,iii,iv - according to uicc tnm classification): II', 'clinical tumor stage (0,i,ii,iii,iv - according to uicc tnm classification): III A', 'clinical tumor stage (0,i,ii,iii,iv - according to uicc tnm classification): III B', 'clinical tumor stage (0,i,ii,iii,iv - according to uicc tnm classification): II A', 'clinical tumor stage (0,i,ii,iii,iv - according to uicc tnm classification): III C'], 8: ['pathological tumor category after neoadjuvant treatment and surgery (0,i,ii,iii,iv - according to uicc tnm classification): 3', 'pathological tumor category after neoadjuvant treatment and surgery (0,i,ii,iii,iv - according to uicc tnm classification): 0', 'pathological tumor category after neoadjuvant treatment and surgery (0,i,ii,iii,iv - according to uicc tnm classification): 2', 'pathological tumor category after neoadjuvant treatment and surgery (0,i,ii,iii,iv - according to uicc tnm classification): 4', 'pathological tumor category after neoadjuvant treatment and surgery (0,i,ii,iii,iv - according to uicc tnm classification): 1'], 9: ['pathological lymphnode status after neoadjuvant treatment and surgery (0,1,2 - according to uicc tnm classification): 1', 'pathological lymphnode status after neoadjuvant treatment and surgery (0,1,2 - according to uicc tnm classification): 0', 'pathological lymphnode status after neoadjuvant treatment and surgery (0,1,2 - according to uicc tnm classification): 2']}

    # Convert the sample characteristics dictionary to a DataFrame
    clinical_data = pd.DataFrame.from_dict(sample_chars_dict, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll analyze the dataset and extract the clinical features as requested:

```python
import pandas as pd
import numpy as np
import os
import json
import gzip
import re

# Let's first look at what files we have in the cohort directory to understand the dataset
print(f"Files in {in_cohort_dir}:")
for file in os.listdir(in_cohort_dir):
    print(f"- {file}")

# Function to extract sample characteristics from series matrix file
def extract_sample_characteristics(file_path):
    sample_char_dict = {}
    current_row = None
    
    with gzip.open(file_path, 'rt') as f:
        lines = []
        in_char_section = False
        samples = []
        
        for line in f:
            if line.startswith('!Sample_geo_accession'):
                samples = line.strip().split('\t')[1:]
                
            elif line.startswith('!Sample_characteristics_ch1'):
                if not in_char_section:
                    in_char_section = True
                
                parts = line.strip().split('\t')
                char_value = parts[0].split('!Sample_characteristics_ch1')[1].strip()
                if char_value:  # If there's a label in the line
                    current_row = len(sample_char_dict)
                    sample_char_dict[current_row] = {'label': char_value, 'values': parts[1:]}
                else:  # If it's a continuation line
                    if current_row is not None:
                        sample_char_dict[current_row]['values'].extend(parts[1:])
            
            elif in_char_section and not line.startswith('!Sample_'):
                in_char_section = False
            
            # Check for data type description
            if line.startswith('!Series_summary'):
                lines.append(line)
            if line.startswith('!Series_title'):
                lines.append(line)
            if line.startswith('!Series_overall_design'):
                lines.append(line)
                
    # Create DataFrame from the dictionary
    df_columns = samples
    df_index = list(range(len(sample_char_dict)))
    df = pd.DataFrame(index=df_index, columns=df_columns)
    
    for row_idx, row_data in sample_char_dict.items():
        for col_idx, value in enumerate(row_data['values']):
            if col_idx < len(df_columns):
                df.iloc[row_idx, col_idx] = value
    
    return df, lines

# Load the matrix file to check if gene expression data is available
matrix_file = os.path.join(in_cohort_dir, "GSE40492_series_matrix.txt.gz")
is_gene_available = False
clinical_data = None
background_info = []

if os.path.exists(matrix_file):
    # Extract sample characteristics from the matrix file
    clinical_data, background_info = extract_sample_characteristics(matrix_file)
    
    # Check for gene expression data by reading the first few lines of the file
    with gzip.open(matrix_file, 'rt') as f:
        # Skip header lines
        for line in f:
            if line.startswith('!series_matrix_table_begin'):
                break
        
        # Read column headers (should be sample IDs)
        header = next(f)
        
        # Check a few data rows to see if they contain gene expression data
        for _ in range(5):
            line = next(f)
            # If the line contains gene IDs and numeric values, it's likely gene expression data
            if re.match(r'^[A-Za-z0-9_-]+\t[-+]?[0-9]*\.?[0-9]+', line):
                is_gene_available = True
                break

# Print background information
print("\nBackground information:")
for line in background_info:
    print(line.strip())

# Print sample characteristics if available
if clinical_data is not None:
    print("\nSample characteristics shape:", clinical_data.shape)
    print("\nSample characteristics preview:")
    print(clinical_data.head(3))
    
    # Print unique values for each row to identify clinical features
    for i in range(len(clinical_data)):
        unique_values = clinical_data.iloc[i].unique()
        print(f"\nRow {i}: {len(unique_values)} unique values:")
        print(unique_values[:10])

# Based on the data exploration, determine clinical features
# For trait (Rectal Cancer - looking for response status)
trait_row = None
age_row = None
gender_row = None

# Scan through the rows to find relevant clinical features
if clinical_data is not None:
    for i in range(len(clinical_data)):
        row_values = [str(val).lower() for val in clinical_data.iloc[i] if not pd.isna(val)]
        row_text = ' '.join(row_values)
        
        # Check for response information (trait)
        if any('responder' in val for val in row_values):
            trait_row = i
        
        # Check for age information
        if any('age' in val for val in row_values):
            age_row = i
        
        # Check for gender/sex information
        if any(gender_term in row_text for gender_term in ['gender', 'sex', 'male', 'female']):
            gender_row = i

print(f"\nIdentified rows - trait: {trait_row}, age: {age_row}, gender: {gender_row}")

# Define conversion functions for clinical features
def convert_trait(value):
    if pd.isna(value):
        return None
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if 'responder' in value:
        return 1  # Responder
    elif 'non-responder' in value:
        return 0  # Non-responder
    else:
        return None

def convert_age(value):
    if pd.isna(value):
        return None
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Try to extract numeric age
    match = re.search(r'(\d+)', value)
    if match:
        return float(match.group(1))
    return None

def convert_gender(value):
    if pd.isna(value):
        return None
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if any(female_term in value for female_term in ['female', 'f', 'woman']):
        return 0
    elif any(male_term in value for male_term in ['male', 'm', 'man']):
        return 1
    else:
        return None

# Validate and save cohort info
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if trait_row is not None and clinical_data is not None:
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    print("\nExtracted clinical features preview:")
    preview = preview_df(clinical_features)
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical features to CSV
    clinical_features.to_csv(out_clinical_data_file)
    print(f"\nClinical


### Step 4: Gene Data Extraction

In [ ]:
# 1. First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print(gene_data.index[:20])


### Step 5: Gene Identifier Review

In [ ]:
# Examining the gene identifiers revealed numeric values ('12', '13', '14', etc.)
# These are clearly not standard human gene symbols (which would be alphanumeric like BRCA1, TP53, etc.)
# These appear to be probe or array identifiers that will need to be mapped to proper gene symbols

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Determine which columns in gene_annotation contain probe IDs and gene symbols
# Looking at the gene_annotation preview, the 'ID' column contains numeric identifiers similar to 
# the gene_data index. For gene symbols, the 'GENE_SYMBOL' column appears to be the appropriate one.
prob_col = 'ID'
gene_col = 'GENE_SYMBOL'

# 2. Get gene mapping dataframe with the two columns
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Convert probe-level measurements to gene-level expression data
# The apply_gene_mapping function splits probe signals equally among mapped genes
# and then sums up contributions to each gene
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Let's check the first few rows of the mapped gene data to verify the transformation
print("Gene expression data after mapping to gene symbols:")
print(gene_data.head())
print(f"Shape of gene expression data: {gene_data.shape}")


### Step 8: Data Normalization and Linking

In [ ]:
# First, we need to re-establish all the required variables and data from previous steps

# 1. Get file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Get background info and clinical data
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# 3. Define the necessary row indices based on analysis of clinical data
# From the sample characteristics dictionary, we can determine:
# Trait: pathological lymph node status is at row 9
# Age: age information is at row 1
# Gender: sex information is at row 2
trait_row = 9
age_row = 1
gender_row = 2

# 4. Define conversion functions for clinical features
def convert_trait(value_str):
    """Convert lymph node status to binary value.
    0 = No positive lymph nodes, 1 = Positive lymph nodes"""
    if value_str is None or 'NA' in value_str:
        return None
    
    # Extract the value after the colon
    if ':' in value_str:
        value = value_str.split(':', 1)[1].strip()
    else:
        value = value_str.strip()
    
    # Status 0 means no positive lymph nodes
    if value == '0':
        return 0
    # Status 1 or 2 means positive lymph nodes
    elif value in ['1', '2']:
        return 1
    else:
        return None

def convert_age(value_str):
    """Convert age to continuous value."""
    if value_str is None or 'NA' in value_str:
        return None
    
    # Extract the value after the colon
    if ':' in value_str:
        value = value_str.split(':', 1)[1].strip()
    else:
        value = value_str.strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value_str):
    """Convert gender to binary value. 0 = female, 1 = male"""
    if value_str is None or 'NA' in value_str:
        return None
    
    # Extract the value after the colon
    if ':' in value_str:
        value = value_str.split(':', 1)[1].strip().lower()
    else:
        value = value_str.strip().lower()
    
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# Reload the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# Get gene annotation data
gene_annotation = get_gene_annotation(soft_file)
prob_col = 'ID'
gene_col = 'GENE_SYMBOL'
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Apply gene mapping to convert probe-level data to gene-level
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Now proceed with the data normalization and linking steps

# 1. Extract clinical features
clinical_features = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row, 
    convert_age=convert_age,
    gender_row=gender_row, 
    convert_gender=convert_gender
)

# Save the clinical features data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains rectal cancer patients with clinical annotations including pathological lymph node status."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")